# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных. 

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*
 
mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL 

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str
    
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)
    
def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)
 
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication 

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])
 
def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])
      
output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, np.float64(2.111520627230723)),
 (1, np.float64(2.111520627230723)),
 (2, np.float64(2.111520627230723)),
 (3, np.float64(2.111520627230723)),
 (4, np.float64(2.111520627230723))]

## Inverted index 

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)
      
def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)
 
def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()
      
def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]
 
def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers
  
def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)
  
  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*
 
e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*
 
flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount 

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)
      
  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):  
    yield (word, 1)
 
def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)
  
# try to set COMBINER=REDUCER and look at the number of values sent over the network 
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None) 
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('it', 18)]),
 (1, [('banana', 2), ('is', 18), ('what', 10)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps
  
  def RECORDREADER(split):
    for value in split:
        yield (value, None)
      
  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])
    
def MAP(value:int, _):
  yield (value, None)
  
def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)
  
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, np.float64(0.015058008454901683)),
   (None, np.float64(0.0166811283706626)),
   (None, np.float64(0.0273949980728333)),
   (None, np.float64(0.07246612838429456)),
   (None, np.float64(0.07674970114408941)),
   (None, np.float64(0.08895159209005588)),
   (None, np.float64(0.09219760891241613)),
   (None, np.float64(0.24122237382260747)),
   (None, np.float64(0.308078887687531)),
   (None, np.float64(0.33650796387656146)),
   (None, np.float64(0.36962699756248973)),
   (None, np.float64(0.3860321748412108)),
   (None, np.float64(0.4137776986643674)),
   (None, np.float64(0.49330973901419706))]),
 (1,
  [(None, np.float64(0.5725607341432004)),
   (None, np.float64(0.5803195185395275)),
   (None, np.float64(0.6079939059863803)),
   (None, np.float64(0.661205842472517)),
   (None, np.float64(0.6932237819796349)),
   (None, np.float64(0.7267957579386604)),
   (None, np.float64(0.7779189378959634)),
   (None, np.float64(0.80290452681718)),
   (None, np.float64(0.8359012313815

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [187]:
import random
from functools import reduce

def MAP(numList):
    return max(numList)

def REDUCE(max1, max2):
    return max(max1, max2)

def RECORDREADER(count):
    return [random.randint(0, 100) for _ in range(count)]

record = RECORDREADER(15)

print(record)
mapped_values = map(MAP, [record]) 
result = reduce(REDUCE, mapped_values)

print("Max value:", result)

[78, 74, 38, 98, 57, 11, 64, 79, 38, 37, 100, 15, 6, 32, 5]
Max value: 100


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [188]:
from typing import Iterator, Tuple

def MAP(_, value: int) -> Iterator[Tuple[None, int]]:
    yield None, value

def REDUCE(_, values: Iterator[int]) -> Iterator[float]:
    values = list(values)  
    yield sum(values) / len(values) if values else 0  

def RECORDREADER(count: int) -> Iterator[Tuple[None, int]]:
    return ((None, random.randint(0, 100)) for _ in range(count))

def MapReduce(data: Iterator[Tuple[None, int]], mapper, reducer) -> Iterator[float]:
    mapped = (pair for key, value in data for pair in mapper(key, value))
    grouped = {}
    
    for key, value in mapped:
        if key not in grouped:
            grouped[key] = []
        grouped[key].append(value)
    
    for key, values in grouped.items():
        yield from reducer(key, values)


data = RECORDREADER(100)
print("Input Data:", [v for _, v in data]) 

result = list(MapReduce(RECORDREADER(100), MAP, REDUCE))
print("Average value:", result[0])



Input Data: [0, 2, 64, 21, 7, 94, 79, 27, 85, 8, 55, 1, 60, 23, 5, 36, 58, 22, 42, 17, 57, 78, 40, 49, 75, 22, 91, 73, 12, 63, 16, 85, 93, 36, 43, 66, 71, 41, 64, 58, 92, 40, 22, 93, 10, 38, 62, 59, 7, 53, 32, 69, 63, 82, 91, 100, 55, 48, 11, 58, 67, 24, 52, 2, 73, 70, 63, 90, 15, 67, 32, 8, 29, 15, 57, 96, 47, 69, 48, 93, 17, 12, 3, 76, 22, 10, 48, 19, 27, 37, 67, 30, 50, 28, 20, 47, 70, 27, 10, 66]
Average value: 49.99


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [189]:
from collections import defaultdict
from typing import List, Tuple

def MAP(num: int) -> Tuple[int, int]:
    return 1, num

def REDUCE(_, values: List[int]) -> float:
    return sum(values) / len(values) if values else 0

def RECORDREADER(count: int) -> List[int]:
    return [random.randint(0, 100) for _ in range(count)]

def group_by_key(iterable: List[Tuple[int, int]]) -> List[Tuple[int, List[int]]]:
    grouped_data = defaultdict(list)
    for key, value in iterable:
        grouped_data[key].append(value)
    return list(grouped_data.items())

def MapReduce(data: List[int]) -> List[float]:
    map_output = list(map(MAP, data))  
    print("MAP output:", map_output)

    shuffle_output = group_by_key(map_output)  
    print("Shuffle output:", shuffle_output)

    reduce_output = [REDUCE(key, values) for key, values in shuffle_output]  
    print("Reduce output:", reduce_output)

    return reduce_output

data = RECORDREADER(100)
print("Input Data:", data)
result = MapReduce(data)



Input Data: [85, 47, 81, 81, 91, 35, 0, 87, 98, 20, 82, 95, 86, 71, 60, 3, 62, 7, 89, 24, 56, 59, 68, 58, 35, 45, 84, 52, 91, 4, 86, 92, 41, 94, 44, 21, 40, 50, 49, 91, 63, 38, 29, 56, 48, 75, 24, 77, 97, 13, 3, 39, 52, 83, 80, 21, 53, 19, 46, 7, 8, 22, 74, 3, 44, 40, 3, 3, 37, 51, 47, 3, 64, 53, 44, 16, 76, 49, 82, 77, 6, 99, 4, 17, 41, 60, 3, 42, 52, 51, 39, 63, 24, 50, 20, 77, 35, 9, 16, 44]
MAP output: [(1, 85), (1, 47), (1, 81), (1, 81), (1, 91), (1, 35), (1, 0), (1, 87), (1, 98), (1, 20), (1, 82), (1, 95), (1, 86), (1, 71), (1, 60), (1, 3), (1, 62), (1, 7), (1, 89), (1, 24), (1, 56), (1, 59), (1, 68), (1, 58), (1, 35), (1, 45), (1, 84), (1, 52), (1, 91), (1, 4), (1, 86), (1, 92), (1, 41), (1, 94), (1, 44), (1, 21), (1, 40), (1, 50), (1, 49), (1, 91), (1, 63), (1, 38), (1, 29), (1, 56), (1, 48), (1, 75), (1, 24), (1, 77), (1, 97), (1, 13), (1, 3), (1, 39), (1, 52), (1, 83), (1, 80), (1, 21), (1, 53), (1, 19), (1, 46), (1, 7), (1, 8), (1, 22), (1, 74), (1, 3), (1, 44), (1, 40), (1,

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [190]:
import numpy as np
from collections import defaultdict

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    grouped_data = defaultdict(list)
    for key, value in iterable:
        grouped_data[key].append(value)
    return list(grouped_data.items())

def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for key, value in map_partition:
            p = partitions[PARTITIONER(key)]
            p.setdefault(key, []).append(value)
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for partition_id, partition in enumerate(partitions)]

def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):

    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    if COMBINER is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    total_values_sent = sum([len(vs) for _, vs in flatten([partition for _, partition in reduce_partitions])])
    print(f"{total_values_sent} key-value pairs were sent over a network.")
    
    return reduce_outputs


d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""

documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for docid, document in enumerate(split):
            for lineid, line in enumerate(document.split('\n')):
                yield ("{}:{}".format(docid, lineid), line)

    split_size = int(np.ceil(len(documents) / maps))
    for i in range(0, len(documents), split_size):
        yield RECORDREADER(documents[i:i + split_size])


def MAP(docId: str, line: str):
    for word in line.split(" "):
        yield (word, word)


def REDUCE(key: str, value: Iterator[str]):
    yield key


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)

partitioned_output = [(partition_id, list(partition)) for partition_id, partition in partitioned_output]

print(partitioned_output)


56 key-value pairs were sent over a network.
[(0, ['', 'a', 'it']), (1, ['banana', 'is', 'what'])]


#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [191]:

def MAP(el_list):
    
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):  
            mapped_result[t].append(t)
    return mapped_result.items()

def REDUCE(mapped_items):
    return [item for sublist in mapped_items for item in sublist]

def C(t):
    return t[0] % 2 == 0

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

record = RECORDREADER(10)
print("Input data:", record)  

part_size = 5
record_partitioned = [record[i:i + part_size] for i in range(0, len(record), part_size)]
print("Partitioned data:", record_partitioned)  

mapped_results = list(map(MAP, record_partitioned))
print("Mapped results:", mapped_results)  

reduced_result = REDUCE(mapped_results)
print("Filtered result:", reduced_result)  


Input data: [(16, 53, 86), (13, 7, 94), (61, 42, 7), (49, 73, 35), (8, 74, 23), (66, 60, 97), (47, 97, 71), (96, 39, 29), (28, 56, 85), (74, 67, 0)]
Partitioned data: [[(16, 53, 86), (13, 7, 94), (61, 42, 7), (49, 73, 35), (8, 74, 23)], [(66, 60, 97), (47, 97, 71), (96, 39, 29), (28, 56, 85), (74, 67, 0)]]
Mapped results: [dict_items([((16, 53, 86), [(16, 53, 86)]), ((8, 74, 23), [(8, 74, 23)])]), dict_items([((66, 60, 97), [(66, 60, 97)]), ((96, 39, 29), [(96, 39, 29)]), ((28, 56, 85), [(28, 56, 85)]), ((74, 67, 0), [(74, 67, 0)])])]
Filtered result: [((16, 53, 86), [(16, 53, 86)]), ((8, 74, 23), [(8, 74, 23)]), ((66, 60, 97), [(66, 60, 97)]), ((96, 39, 29), [(96, 39, 29)]), ((28, 56, 85), [(28, 56, 85)]), ((74, 67, 0), [(74, 67, 0)])]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [192]:
from typing import Iterator, NamedTuple

S = {25, 90, 69}

def MAP(t):
    filtered = tuple(el for el in t if el in S)  
    return (filtered, filtered)  

def REDUCE(key, values: Iterator[NamedTuple]):
    return key, key 

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]  
    return t.items()  

record = RECORDREADER(5)
print("Input records:", record)

map_output = list(map(MAP, RECORDREADER(100)))  
print("MAP output:", map_output)

shuffle_output = group_by_key(map(MAP, RECORDREADER(100)))  
print("Shuffle output:", list(shuffle_output))

reduce_output = list(map(lambda x: REDUCE(*x)[0], list(shuffle_output)))  
print("Reduce output:", reduce_output)


Input records: [(80, 57, 85), (9, 97, 98), (25, 100, 52), (63, 28, 68), (15, 41, 55)]
MAP output: [((), ()), ((), ()), ((), ()), ((90,), (90,)), ((), ()), ((), ()), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((), ()), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((), ()), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((25,), (25,)), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((), ()), ((69,), (69,)), ((25, 25), (25, 25)), ((), ()), ((90,), (90,)), ((), ()), ((), ()), (()

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [193]:
def MAP(t):
    return (t, t)  

def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)  

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]  

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:  
        t[k2] = t.get(k2, []) + [v2]  
    return t.items()  

record = RECORDREADER(5)  
print("Input records:", record)

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))  
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)  
shuffle_output = list(shuffle_output)  
print("Shuffle output:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))  
print("Reduce output:", reduce_output)

Input records: [(31, 90, 99), (13, 60, 34), (55, 22, 33), (69, 31, 64), (86, 54, 74)]
MAP output: [((59, 42, 27), (59, 42, 27)), ((57, 12, 0), (57, 12, 0)), ((78, 18, 26), (78, 18, 26)), ((9, 98, 84), (9, 98, 84)), ((88, 5, 58), (88, 5, 58)), ((27, 24, 66), (27, 24, 66)), ((17, 87, 53), (17, 87, 53)), ((48, 46, 35), (48, 46, 35)), ((10, 52, 13), (10, 52, 13)), ((68, 28, 61), (68, 28, 61)), ((89, 72, 80), (89, 72, 80)), ((52, 91, 33), (52, 91, 33)), ((94, 43, 91), (94, 43, 91)), ((99, 47, 44), (99, 47, 44)), ((34, 92, 58), (34, 92, 58)), ((11, 59, 22), (11, 59, 22)), ((69, 7, 63), (69, 7, 63)), ((28, 7, 45), (28, 7, 45)), ((35, 99, 21), (35, 99, 21)), ((31, 59, 62), (31, 59, 62)), ((7, 82, 39), (7, 82, 39)), ((54, 66, 96), (54, 66, 96)), ((77, 97, 77), (77, 97, 77)), ((27, 83, 40), (27, 83, 40)), ((90, 54, 0), (90, 54, 0)), ((70, 46, 14), (70, 46, 14)), ((35, 94, 59), (35, 94, 59)), ((100, 29, 100), (100, 29, 100)), ((28, 52, 43), (28, 52, 43)), ((56, 33, 32), (56, 33, 32)), ((67, 34, 5

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [201]:

def MAP(t):
    return (t, t)

def REDUCE(key, values: Iterator[NamedTuple]):
    if len(values) == 2:
        return (key, key)

def generate_records(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for _ in range(count)]

def group_by_key(data):
    grouped = {}
    for (key, value) in data:
        grouped.setdefault(key, []).append(value)  
    return grouped.items()

records = generate_records(100)
print("Generated records:", records)

mapped_data = [MAP(record) for record in records]
print("Mapped data:", mapped_data)

grouped_data = list(group_by_key(mapped_data))
print("Grouped data:", grouped_data)

reduced_data = [result[0] for result in map(lambda x: REDUCE(*x), grouped_data) if result is not None]
print("Reduced data:", reduced_data)


Generated records: [(2, 3), (3, 0), (0, 1), (2, 2), (3, 3), (1, 0), (2, 3), (2, 2), (0, 1), (2, 2), (0, 2), (3, 1), (2, 3), (2, 1), (0, 1), (0, 3), (2, 2), (3, 2), (0, 2), (1, 2), (1, 0), (2, 2), (0, 1), (1, 0), (1, 2), (1, 0), (0, 0), (3, 1), (3, 2), (2, 0), (3, 0), (0, 1), (2, 2), (0, 3), (0, 0), (0, 2), (1, 1), (1, 0), (1, 0), (3, 0), (1, 1), (3, 2), (0, 3), (3, 2), (0, 0), (0, 2), (0, 2), (3, 1), (0, 0), (3, 2), (0, 3), (3, 2), (1, 0), (2, 3), (3, 0), (2, 2), (1, 0), (0, 3), (3, 1), (3, 0), (2, 3), (0, 2), (0, 1), (1, 3), (2, 0), (0, 2), (3, 0), (3, 1), (3, 0), (3, 3), (3, 1), (2, 3), (0, 2), (3, 0), (2, 3), (0, 3), (2, 3), (0, 1), (0, 1), (1, 3), (0, 0), (0, 3), (1, 3), (2, 0), (2, 0), (2, 0), (3, 1), (2, 1), (2, 3), (1, 2), (1, 0), (0, 3), (2, 1), (2, 3), (0, 0), (1, 0), (3, 1), (0, 0), (2, 3), (3, 2)]
Mapped data: [((2, 3), (2, 3)), ((3, 0), (3, 0)), ((0, 1), (0, 1)), ((2, 2), (2, 2)), ((3, 3), (3, 3)), ((1, 0), (1, 0)), ((2, 3), (2, 3)), ((2, 2), (2, 2)), ((0, 1), (0, 1)), ((2,

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [202]:

rels = [1, 2]  

class Tuple: 
    def __init__(self, data: tuple, rel_id: int):
        self.data = data
        self.rel_id = rel_id

def get_random_tuple(count):
    data = tuple((random.randint(0, 3), random.randint(0, 3)) for _ in range(count))  
    rel_id = random.choice(rels)  
    return Tuple(data, rel_id)

def RECORDREADER(count):
    return [get_random_tuple(3) for _ in range(count)]

def MAP(t: Tuple):
    return (t.data, t.rel_id)

def REDUCE(key, values: Iterator[NamedTuple]):
    if values == [rels[0]]:  
        return (key, key)

def group_by_key(iterable):
    grouped = {}
    for k2, v2 in iterable:
        grouped.setdefault(k2, []).append(v2)  
    return grouped.items()


record = RECORDREADER(100)


print("Input records:", [(t.data, t.rel_id) for t in record])

map_output = [MAP(t) for t in record]  
print("MAP output:", map_output)

shuffle_output = list(group_by_key(map_output))
print("Shuffle output:", shuffle_output)

reduce_output = [result[0] for result in map(lambda x: REDUCE(*x), shuffle_output) if result is not None]
print("Reduce output:", reduce_output)


Input records: [(((2, 0), (1, 3), (3, 1)), 1), (((2, 3), (3, 1), (0, 3)), 2), (((0, 2), (3, 2), (0, 2)), 2), (((2, 0), (2, 2), (1, 1)), 1), (((3, 1), (3, 1), (0, 3)), 1), (((1, 0), (3, 0), (2, 2)), 2), (((1, 0), (2, 1), (3, 2)), 2), (((1, 1), (3, 0), (3, 0)), 1), (((0, 1), (2, 2), (2, 1)), 1), (((1, 1), (1, 3), (1, 0)), 1), (((1, 1), (0, 3), (2, 0)), 1), (((1, 2), (3, 2), (2, 3)), 1), (((3, 0), (3, 2), (0, 1)), 1), (((0, 0), (3, 3), (3, 2)), 1), (((0, 2), (1, 0), (0, 3)), 2), (((0, 1), (2, 1), (0, 1)), 2), (((2, 3), (1, 2), (2, 0)), 2), (((3, 3), (3, 2), (1, 1)), 1), (((2, 2), (2, 2), (2, 3)), 1), (((0, 2), (2, 2), (2, 1)), 2), (((3, 3), (3, 3), (1, 0)), 2), (((2, 3), (2, 3), (0, 3)), 1), (((1, 0), (2, 3), (2, 0)), 2), (((1, 0), (1, 0), (0, 1)), 1), (((1, 2), (2, 1), (3, 3)), 2), (((1, 3), (3, 0), (1, 0)), 1), (((2, 0), (1, 0), (1, 0)), 2), (((2, 0), (0, 1), (2, 2)), 2), (((0, 0), (1, 1), (1, 3)), 1), (((2, 3), (3, 3), (0, 3)), 1), (((1, 3), (2, 2), (3, 2)), 2), (((0, 1), (3, 2), (0, 2

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [ ]:

rels = [1, 2]  

class Tuple:
    def __init__(self, data: tuple, rel_id: int):
        self.data = data
        self.rel_id = rel_id

def get_random_tuple():  
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)

def RECORDREADER(count):
    return [get_random_tuple() for _ in range(count)]

def MAP(t: Tuple):
    return (t.data[1], (t.rel_id, t.data[0])) if t.rel_id == rels[0] else (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):
    return [(v[0], key, v[1]) for v in values]

def group_by_key(iterable):
    grouped = {}
    for k, v in iterable:
        grouped.setdefault(k, []).append(v)
    return grouped.items()

record = RECORDREADER(100)
print("Input records:", [(t.data, t.rel_id) for t in record])

map_output = list(map(MAP, record))
print("MAP output:", map_output)

shuffle_output = list(group_by_key(map_output))
print("Shuffle output:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print("Reduce output:", reduce_output)


Input records: [((1, 2), 2), ((0, 3), 2), ((2, 1), 2), ((3, 1), 1), ((0, 2), 2), ((1, 3), 2), ((1, 2), 1), ((1, 2), 1), ((0, 0), 1), ((0, 3), 1), ((3, 1), 1), ((2, 1), 2), ((3, 0), 2), ((1, 2), 2), ((3, 1), 1), ((2, 1), 1), ((3, 0), 1), ((2, 3), 2), ((0, 2), 2), ((1, 1), 1), ((3, 0), 1), ((0, 1), 1), ((0, 3), 2), ((3, 3), 2), ((0, 2), 1), ((3, 3), 1), ((1, 0), 1), ((1, 1), 2), ((3, 1), 1), ((1, 2), 2), ((1, 0), 2), ((3, 2), 2), ((3, 2), 1), ((3, 2), 2), ((1, 0), 2), ((0, 1), 2), ((2, 2), 2), ((0, 0), 1), ((2, 3), 1), ((2, 3), 2), ((0, 0), 2), ((2, 2), 1), ((0, 0), 2), ((2, 2), 2), ((1, 0), 1), ((3, 1), 1), ((3, 0), 2), ((0, 1), 1), ((0, 2), 1), ((1, 2), 1), ((1, 2), 2), ((0, 1), 1), ((2, 2), 1), ((2, 3), 2), ((3, 0), 2), ((3, 2), 1), ((3, 3), 1), ((3, 2), 1), ((3, 1), 1), ((3, 1), 1), ((0, 2), 2), ((0, 0), 2), ((2, 0), 1), ((1, 0), 2), ((2, 3), 2), ((1, 0), 1), ((2, 0), 1), ((3, 2), 1), ((3, 3), 2), ((1, 0), 1), ((1, 3), 1), ((3, 3), 2), ((3, 0), 2), ((2, 2), 1), ((2, 0), 1), ((3, 0), 

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [213]:
from typing import List, Tuple

def generate_data_point() -> Tuple[int, int, int]:
    return tuple(random.randint(0, 3) for _ in range(3))

def create_dataset(size: int) -> List[Tuple[int, int, int]]:
    return [generate_data_point() for _ in range(size)]

def transform(data_point: Tuple[int, int, int]) -> Tuple[int, int]:
    return data_point[:2]

def aggregate(key: Tuple[int, int], values: List[int]) -> Tuple[Tuple[int, int], int]:
    return key, sum(values)

def group_data(data: List[Tuple[int, int]]) -> dict:
    grouped = defaultdict(list)
    for key, value in data:
        grouped[key].append(value)
    return dict(grouped)


record = create_dataset(100)
print("Input record:", record)  

transformed_data = [transform(point) for point in record]
print("Transformed data:", transformed_data)

grouped_data = group_data(transformed_data)
print("Grouped data:", grouped_data)

aggregated_results = [aggregate(key, values) for key, values in grouped_data.items()]
print("Aggregated results:", aggregated_results)


Input record: [(2, 3, 3), (0, 0, 2), (2, 3, 0), (3, 0, 2), (2, 1, 0), (3, 1, 0), (2, 0, 1), (2, 0, 0), (3, 0, 2), (2, 3, 0), (0, 2, 1), (3, 3, 2), (0, 3, 0), (0, 3, 1), (1, 1, 0), (2, 1, 3), (3, 1, 1), (2, 3, 2), (1, 2, 3), (2, 2, 0), (1, 2, 2), (0, 0, 1), (1, 2, 0), (2, 3, 0), (2, 1, 3), (0, 3, 1), (0, 2, 0), (3, 3, 1), (0, 2, 3), (3, 0, 2), (0, 3, 2), (3, 3, 1), (1, 0, 1), (0, 0, 0), (3, 1, 2), (0, 3, 0), (1, 0, 2), (0, 1, 3), (1, 1, 1), (2, 0, 0), (2, 2, 0), (1, 0, 3), (0, 3, 1), (1, 3, 1), (0, 2, 1), (1, 1, 0), (3, 3, 2), (1, 1, 1), (2, 2, 3), (1, 3, 1), (3, 0, 0), (1, 1, 1), (3, 0, 2), (0, 1, 3), (1, 2, 3), (2, 1, 0), (1, 1, 0), (1, 0, 0), (0, 1, 1), (0, 2, 1), (1, 3, 0), (3, 2, 1), (0, 2, 2), (3, 2, 2), (3, 1, 2), (0, 1, 2), (1, 0, 0), (3, 0, 1), (1, 3, 2), (1, 0, 0), (0, 1, 0), (3, 3, 2), (0, 1, 1), (3, 3, 1), (1, 1, 3), (3, 1, 2), (3, 2, 0), (3, 3, 3), (2, 0, 2), (0, 1, 0), (1, 2, 3), (3, 3, 1), (3, 0, 0), (0, 2, 3), (1, 2, 2), (2, 1, 3), (3, 0, 1), (2, 3, 1), (3, 1, 3), (2, 0,

# 

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$. 





In [173]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [174]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])
      
def MAP(k1, v1):
  (j, k) = k1
  w = v1
  # solution code that yield(k2,v2) pairs
  for i in range(small_mat.shape[0]):
        yield (i, k), w * small_mat[i][j]
  
def REDUCE(key, values):
  (i, k) = key
  # solution code that yield(k3,v3) pairs
  el_value = sum(values)
  yield key, el_value



Проверьте своё решение

In [175]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat) 
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true


True

In [176]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [209]:

I, J, K = 2, 3, 40

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested):
    for iterable in nested:
        yield from iterable

def groupbykey(iterable):
    result = {}
    for k, v in iterable:
        result.setdefault(k, []).append(v)
    return result.items()

def RECORDREADER():
    yield [((0, i, j), small_mat[i, j]) for i in range(I) for j in range(J)]
    yield [((1, j, k), big_mat[j, k]) for j in range(J) for k in range(K)]

def MAP_JOIN(k1, v1):
    mat_num, i, j = k1
    w = v1
    if mat_num == 0:
        yield (j, (mat_num, i, w))
    else:
        yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
    from_first_mat = [v for v in values if v[0] == 0]
    from_second_mat = [v for v in values if v[0] == 1]
    for f in from_first_mat:
        for s in from_second_mat:
            yield ((f[1], s[1]), f[2] * s[2])

def MAP_MUL(k1, v1):
    yield k1, v1

def REDUCE_MUL(key, values):
    yield key, sum(values)

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I_max = max(i for ((i, _), _) in reduce_output) + 1
    K_max = max(k for ((_, k), _) in reduce_output) + 1
    mat = np.zeros((I_max, K_max))
    for (i, k), value in reduce_output:
        mat[i, k] = value
    return mat

def MapReduce(RECORDREADER, MAP, REDUCE):
    mapped = flatten(map(lambda kv: MAP(*kv), flatten(RECORDREADER())))
    reduced = flatten(map(lambda kv: REDUCE(*kv), groupbykey(mapped)))
    return reduced

joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(lambda: [joined], MAP_MUL, REDUCE_MUL)

assert np.allclose(reference_solution, asmatrix(solution)), "False"
print("True")


True


Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER. 

In [207]:

I, J, K = 2, 3, 40

maps = 4
reducers = 2

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
    for iterable in nested_iterable:
        yield from iterable

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
    partitions = defaultdict(dict)
    for map_partition in map_partitions:
        for k2, v2 in map_partition:
            partitions[PARTITIONER(k2)][k2] = partitions[PARTITIONER(k2)].get(k2, []) + [v2]
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for partition_id, partition in partitions.items()]

def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    if COMBINER:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

    print(f"{sum(len(vs) for _, vs in flatten([partition for _, partition in reduce_partitions]))} key-value pairs were sent over a network.")
    return reduce_outputs

def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I_max = max(i for ((i, _), _) in reduce_output) + 1
    K_max = max(k for ((_, k), _) in reduce_output) + 1
    mat = np.zeros((I_max, K_max))
    for (i, k), value in reduce_output:
        mat[i, k] = value
    return mat

def INPUTFORMAT():
    first_mat = [((0, i, j), val) for i in range(small_mat.shape[0]) for j in range(small_mat.shape[1]) for val in [small_mat[i, j]]]
    second_mat = [((1, j, k), val) for j in range(big_mat.shape[0]) for k in range(big_mat.shape[1]) for val in [big_mat[j, k]]]
    yield first_mat
    yield second_mat

def MAP_JOIN(k1, v1):
    mat_num, i, j = k1
    if mat_num == 0:
        yield j, (mat_num, i, v1)
    else:
        yield i, (mat_num, j, v1)

def REDUCE_JOIN(key, values):
    from_first_mat = [v for v in values if v[0] == 0]
    from_second_mat = [v for v in values if v[0] == 1]
    for f in from_first_mat:
        for s in from_second_mat:
            yield (f[1], s[1]), f[2] * s[2]

def GET_JOINED():
    for j in joined:
        yield j[1]

def MAP_MUL(k1, v1):
    yield k1, v1

def REDUCE_MUL(key, values):
    yield key, sum(values)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN)
joined = [(partition_id, list(partition)) for partition_id, partition in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL)
pre_result = [(partition_id, list(partition)) for partition_id, partition in mul_output]
print(pre_result)

solution = [v for p in pre_result for v in p[1]]
print(solution)

assert np.allclose(reference_solution, asmatrix(solution)), "False"
print("True")


126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.09559004128890893)), ((0, 1), np.float64(0.0018310064834232902)), ((0, 2), np.float64(0.03427477929679909)), ((0, 3), np.float64(0.07684087191870846)), ((0, 4), np.float64(0.10796937043966663)), ((0, 5), np.float64(0.12546296664915704)), ((0, 6), np.float64(0.13929405844492881)), ((0, 7), np.float64(0.1398013262022728)), ((0, 8), np.float64(0.10515770041039041)), ((0, 9), np.float64(0.03315979492361304)), ((0, 10), np.float64(0.07338737412485595)), ((0, 11), np.float64(0.013810471954323384)), ((0, 12), np.float64(0.023348237639219248)), ((0, 13), np.float64(0.018366669712473437)), ((0, 14), np.float64(0.07751243307119034)), ((0, 15), np.float64(0.017581844078193555)), ((0, 16), np.float64(0.07547098082561131)), ((0, 17), np.float64(0.09282144514536404)), ((0, 18), np.float64(0.09397923348863989)), ((0, 19), np.float64(0.12282191760217095)), ((0, 20), np.float64(0.09651712163579214)), ((0, 21), np.float64(0.059113

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [208]:
I, J, K = 2, 3, 40

maps = 3
reducers = 2

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

def INPUTFORMAT():
  first_mat = []

  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))  

  global maps
  split_size = int(np.ceil(len(first_mat)/maps))

  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))  

  split_size = int(np.ceil(len(second_mat)/maps))

  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]

  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0

  for v in values:
    res_val += v
  yield (key, res_val)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = [v for p in pre_result for v in p[1]]

print(solution)

assert np.allclose(reference_solution, asmatrix(solution)), "False"
print("True")

126 key-value pairs were sent over a network.
[(0, [((0, 0), np.float64(0.7687104556926646)), ((0, 1), np.float64(0.5450446527837265)), ((0, 2), np.float64(0.4478192150630807)), ((0, 3), np.float64(0.39847865358656)), ((0, 4), np.float64(0.555494110321492)), ((0, 5), np.float64(0.29649677305565664)), ((0, 6), np.float64(0.058360367844883965)), ((0, 7), np.float64(0.26908285803762483)), ((0, 8), np.float64(0.05296038717613254)), ((0, 9), np.float64(0.44920597589191447)), ((0, 10), np.float64(0.6682056343472671)), ((0, 11), np.float64(0.14319296251399907)), ((0, 12), np.float64(0.13925114816149486)), ((0, 13), np.float64(0.2500671947920897)), ((0, 14), np.float64(0.41410167607979664)), ((0, 15), np.float64(0.14509319472691318)), ((0, 16), np.float64(0.5661727139092083)), ((0, 17), np.float64(0.4779861445022997)), ((0, 18), np.float64(0.22222482336341515)), ((0, 19), np.float64(0.5319651902902233)), ((0, 20), np.float64(0.18021971143518778)), ((0, 21), np.float64(0.5106733838667865)), ((0